In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Load a biomedical language model (BioGPT).
# You can swap in another clinical LLM if available.
model_name = "microsoft/BioGPT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Set up the text generation pipeline for clinical prompts
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    do_sample=True,
    temperature=0.3
)

# Define a sample patient case to analyze
patient_case = (
    "67-year-old female with newly diagnosed Type 2 Diabetes. "
    "HbA1c: 8.2%, BMI: 32, eGFR: 65 mL/min. "
    "Comorbidities: Hypertension. No cardiovascular disease. "
    "Current medications: Lisinopril 10mg daily.\n"
    "Based on current clinical guidelines, provide evidence-based treatment recommendations:"
)

# Print a heading for clarity
print("MEDICAL LLM TREATMENT RECOMMENDATION")
print("=" * 50)

# Generate a response from the biomedical LLM
response = generator(patient_case, max_new_tokens=200, num_return_sequences=1)

# Extract and display only the LLM's generated text (excluding the prompt)
print(response[0]['generated_text'][len(patient_case):].strip())

# Always include a disclaimer for AI-generated healthcare content
print("\n⚠️ AI-generated content for demonstration only. Not for clinical use.")


In [ ]:
from transformers import pipeline

# Set up a medical question-answering pipeline using a pre-trained model
qa_pipeline = pipeline(
    "question-answering",
    model=" dmis-lab/biobert-base-cased-v1.1
    tokenizer=" dmis-lab/biobert-base-cased-v1.1"
)

# Medical knowledge base for the chatbot (expand as needed)
medical_knowledge = {
    "diabetes": (
        "Type 2 diabetes is a condition where blood sugar levels are too high due to "
        "insulin resistance. Common symptoms include increased thirst, frequent urination, "
        "fatigue, and blurred vision. Management includes healthy eating, regular exercise, "
        "weight management, and medications like metformin as prescribed by your doctor."
    ),
    "hypertension": (
        "High blood pressure, or hypertension, is when blood pressure is consistently too high. "
        "It is called the 'silent killer' because it often has no symptoms. Management includes "
        "reducing salt, regular exercise, healthy weight, limiting alcohol, and taking prescribed "
        "medications such as ACE inhibitors or diuretics."
    )
}

def medical_chatbot_response(patient_question):
    """Generate a response using the QA pipeline and a simple keyword match for topics."""
    # Emergency check
    emergency_terms = ["chest pain", "difficulty breathing", "severe headache", "stroke"]
    if any(term in patient_question.lower() for term in emergency_terms):
        return " This sounds like a medical emergency. Please call 911 or go to the nearest emergency room immediately."

    # Simple keyword/topic match
    for topic, context in medical_knowledge.items():
        if topic in patient_question.lower():
            answer = qa_pipeline(question=patient_question, context=context)
            return (
                f"{answer['answer']}\n\n"
                " This is general health education. Always consult your healthcare provider for personalized medical advice."
            )

    # No match found
    return (
        "I'm sorry, I don't have information about this topic. "
        "Please consult your healthcare provider for medical guidance."
    )

# Test the simplified medical chatbot
print("SIMPLE MEDICAL CHATBOT")
print("=" * 40)

test_questions = [
    "What are the symptoms of diabetes?",
    "How can I manage my high blood pressure?",
    "I'm having severe chest pain"
]

for question in test_questions:
    print(f"\nPatient: {question}")
    response = medical_chatbot_response(question)
    print(f"Chatbot: {response}")

print("\n⚠️ AI-generated content for demonstration only. Not for clinical use.")


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Load a summarization model (BART fine-tuned for summarization)
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up summarization pipeline
summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    max_length=120,
    min_length=40,
    do_sample=False
)

# Sample clinical note
clinical_note = """
72-year-old male presents with acute chest pain and shortness of breath.
Sudden onset, severe substernal pain radiating to left arm and jaw, diaphoresis, and nausea.
History of hypertension, diabetes, hyperlipidemia. Medications: metformin, lisinopril, atorvastatin.
Vitals: BP 160/95, HR 110, RR 22, O2 sat 94%. Exam: tachycardia, no murmurs, crackles in lungs, no edema.
Assessment: Acute STEMI, heart failure with pulmonary edema.
Plan: Cardiac cath, dual antiplatelet therapy, heparin, furosemide, cardiac ICU.
"""

# Generate summary
print("CLINICAL NOTE SUMMARIZATION")
print("=" * 50)

summary = summarizer(clinical_note, max_length=80, min_length=30)
print("Automated Summary:")
print(summary[0]['summary_text'])

print("\n⚠️ AI-generated summary for demonstration only. Requires physician review.")


In [ ]:
from transformers import pipeline
import re

# Load a biomedical NER model (BioBERT-based)
ner_pipeline = pipeline(
    "ner",
    model="d4data/biomedical-ner-all",
    tokenizer="d4data/biomedical-ner-all",
    aggregation_strategy="simple"
)

# Known drug-adverse event associations
known_drug_events = {
    "metformin": ["nausea", "diarrhea", "lactic acidosis", "vitamin b12 deficiency"],
    "atorvastatin": ["muscle pain", "myopathy", "rhabdomyolysis", "liver toxicity"],
    "lisinopril": ["dry cough", "hyperkalemia", "angioedema", "hypotension"],
    "warfarin": ["bleeding", "bruising", "hemorrhage", "skin necrosis"],
    "prednisone": ["weight gain", "mood changes", "osteoporosis", "diabetes"]
}

def extract_entities(text):
    """Extract drugs and adverse events from clinical text using NER."""
    entities = ner_pipeline(text)
    drugs = [e['word'].lower() for e in entities if e['word'].lower() in known_drug_events]
    events = [
        e['word'].lower() for e in entities
        if any(e['word'].lower() in events for events in known_drug_events.values())
    ]
    return list(set(drugs)), list(set(events))

def temporal_evidence(text, drug, event):
    """Simple check for temporal or causal phrases."""
    patterns = [
        rf"{drug}.*?after.*?{event}", rf"{event}.*?after.*?{drug}",
        rf"{drug}.*?caused.*?{event}", rf"{event}.*?due to.*?{drug}",
        rf"since starting {drug}.*?{event}", rf"following {drug}.*?{event}"
    ]
    text = text.lower()
    return any(re.search(p, text) for p in patterns)

def detect_ade(text):
    """Detect potential ADEs in a clinical note."""
    drugs, events = extract_entities(text)
    results = []
    for drug in drugs:
        for event in events:
            known = event in known_drug_events.get(drug, [])
            temporal = temporal_evidence(text, drug, event)
            conf = 0.2 + (0.5 if known else 0) + (0.3 if temporal else 0)
            if conf >= 0.4:
                results.append({
                    'drug': drug,
                    'event': event,
                    'confidence': round(conf, 2),
                    'known': known,
                    'temporal': temporal
                })
    return sorted(results, key=lambda x: x['confidence'], reverse=True)

# Example test cases
test_cases = [
    "Patient started on atorvastatin. Developed muscle pain after starting medication.",
    "Metformin therapy initiated. Patient reports diarrhea.",
    "No bleeding complications with warfarin.",
    "Lisinopril caused dry cough."
]

print("ADE DETECTION DEMO")
print("=" * 40)
for i, text in enumerate(test_cases, 1):
    print(f"\nCase {i}: {text}")
    ades = detect_ade(text)
    if ades:
        for ade in ades:
            print(f"- Drug: {ade['drug']} | Event: {ade['event']} | Confidence: {ade['confidence']} | Known: {ade['known']} | Temporal: {ade['temporal']}")
    else:
        print("No significant ADEs detected.")

print("\n⚠️ Demo only. AI-generated content requires clinical review.")
